In [ ]:
!pip -q install ultralytics

# 0) 환경 설정 및 필요한 라이브러리 임포트
# Colab에서 실행할 경우 아래 코드가 자동으로 Google Drive를 마운트합니다.
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
except:
    IN_COLAB = False

import os
# --- 데이터 및 모델 경로 설정 ---
# Colab 환경: '/content/drive/MyDrive/...'
# 로컬 환경: 현재 폴더('.') 기준
BASE_DIR = '/content/drive/MyDrive/coins' if IN_COLAB else '.'
WEIGHTS_DIR = '/content/drive/MyDrive/yolov8_project/coins_cls/weights' if IN_COLAB else os.path.join(BASE_DIR, 'weights')

TEST_IMG_PATH = os.path.join(BASE_DIR, 'test_scene.jpg')
BEST_MODEL_PATH = os.path.join(WEIGHTS_DIR, 'best.pt')
LAST_MODEL_PATH = os.path.join(WEIGHTS_DIR, 'last.pt')
DATASET_PATH = os.path.join(BASE_DIR, 'data')

import cv2
import numpy as np
from ultralytics import YOLO
from collections import Counter

# 원본 이미지 넣기
img_path = TEST_IMG_PATH

# 1) 이미지 받기: 여러 동전이 있는 이미지를 로드
def load_image(img_path: str):   # 이미지 파일의 경로(path) 문자열을 받는다
    src = cv2.imread(img_path, cv2.IMREAD_COLOR) # 경로에 있는 이미지를 읽어서 컬러로 받는다
    if src is None:
        raise FileNotFoundError(f"이미지 읽기 실패: {img_path}") # 이미지 로드가 실패시 오류 메세지
    return src

# 2) 받은 이미지에서 동전을 찾기 쉽도록 전처리
def preprocess_for_detection(src: np.ndarray): # OpenCV로 읽은 이미지(배열)을 입력으로 받는다

    # 받은 이미지를 그레이스케일(명암도) 이미지로 변환
    # 단체사진에서 개별 동전 검출은 색보다 밝기 차이와 경계 구분이 중요하기에 이진화(흑/백 분리)가 쉬워지게 변환
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

    # 위의 그레이 이미지에 가우시안 블러(흐림) 적용
    # 동전 표면엔 글자, 무늬같은 작은 요철 등이 많음
    # 가우시안 블러는 이런 작은 디테일을 뭉개서 동전 영역을 한 덩어리로 유지해서 이진화하면 동전 내부가 점점이 끊기는 일을 방지
    # (7,7): 블러 커널 크기 (클수록 더 흐려짐) 2,2: 표준편차(흐림 강도)
    gray = cv2.GaussianBlur(gray, (7, 7), 2, 2)

    # 이후 이어질 도형 검출을 더 안정적으로 할 수 있게 이진화(그레이 이미지보다 이진화가 훨씬 안정적)
    # THRESH_BINARY : 밝으면 흰색(255)/어두우면 검정(0)
    # THRESH_OTSU : 임계값을 임의로 정해서 픽셀들을 두 부류로 나누고 두 부류의 밝기 분포를 반복해서 구하고
    # > 그 다음 두 부류의 밝기 분포를 가장 균일하게 하는(그룹 내부 분산(흩어짐)은 최소, 그룹 간 차이는 최대) 경계 값을 선택
    # >>사진마다 조명/그림자/밝기가 달라서 임계값을 임의로 고정하면 문제가 생길 수 있어서 사용
    flag = cv2.THRESH_BINARY + cv2.THRESH_OTSU

    # 그레이 이미지를 흑/백(0/255) 으로 바꾸는 이진화 수행
    _, th = cv2.threshold(gray, 0, 255, flag)

    # 작은 잡영 제거, 작은 점/구멍 같은 잡음을 없애거나,끊긴 부분을 이어붙이는 필터링
    # OPEN 방식은 침식(Erode) → 팽창(Dilate) 순, 커널을 이미지의 왼쪽 위부터 끝까지 한칸씩 쭉 움직이며 계산
    # 침식(Erode): 커널(예: 3×3)을 어떤 위치에 올려놨을 때 커널이 덮는 영역이 전부 흰색(255)이어야 그 중심 픽셀이 흰색으로 남음, 하나라도 검정(0)이 섞이면 중심 픽셀은 검정으로 바뀜
    # 팽창(Dilate): 커널이 덮는 영역 안에 흰색이 하나라도 있으면 그 중심 픽셀을 흰색으로 만듦, 침식으로 줄어든 큰 덩어리를 다시 복구
    th = cv2.morphologyEx(th, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    return th

# 3) 이진 이미지(th)에서 동전 후보 덩어리(contour)를 찾고, 각 덩어리들의 최소 외접원들의 중심,반지름 리스트(cirlcles)을 반환한다.
def detect_coin_circles(th: np.ndarray, min_radius: int = 25):

    # 위에서 받은 흑백 이미지에서 흰색(255)으로 연결된 영역들을 찾고,그 영역의 테두리(윤곽선) 좌표들을 리스트로 만듬.
    # findContours(): 이진 이미지 전체를 스캔하여 시작 경계 픽셀을 하나 잡고 그 픽셀 주변 8칸을 일정한 순서(예: 시계방향)로 훑어서 다음 경계 픽셀을 찾고 그걸 계속 반복해서 결국 한 바퀴 돌아 원점으로 돌아오면 종료

    # 픽셀 시작점 선택: 1. 픽셀이 흰색(255) / 2. 이전에 이미 contour로 처리된 물체에 속하지 않음 / 3. 경계 픽셀 판정 조건 만족
    # 경계 픽셀 판정 조건: 어떤 흰색 픽셀 P가 있을 때 주변(8방향) 이웃 중에 검정(0)이 하나라도 있으면 P는 경계(테두리) 픽셀이라 판단

    # RETR_EXTERNAL: 가장 바깥 윤곽선 가져오기/ CHAIN_APPROX_SIMPLE: 픽셀 격자 내 직선으로 이어지는 구간의 중간 점들을 다 버리고 방향이 바뀌는 꼭짓점(코너)만 남기는 식으로 압축
    # _: 원래는 hierarchy, 각 contour마다 4개의 인덱스(다음 윤곽선(next)/이전 윤곽선(prev)/첫 자식(child)/부모(parent))를 가지는 배열로 윤곽선들 사이의 포함 관계 정보를 담는다
    # hierarchy는 여기선 사용하지 않기에 "contours, _ = ..."" 같은 형태로 받지 않겠다고 한 것

    contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 이제 위에서 만든 윤곽선 리스트(contours)를 가지고 최소 외접원(minEnclosingCircle(c))을 만듬
    circles = [cv2.minEnclosingCircle(c) for c in contours]

    # 1. 기존 circles (for center, radius in circles) 를 꺼내서
    # 2. radius > min_radius 조건을 만족하는 것만 남긴다
    circles = [(center, int(radius)) for center, radius in circles if radius > min_radius]

    # circles = [((x, y), r), ((x, y), r), ...] 리스트를
    # x좌표가 작은 동전부터 (왼쪽 → 오른쪽) x가 같으면 y좌표가 작은 동전부터 (위 → 아래) 로 정렬
    circles.sort(key=lambda x: (x[0][0], x[0][1]))
    return circles


# 4) 이미지에서 위에서 받은 circles 리스트를 기준으로 동전을 하나씩 잘라낸다.
# crop_scale: 동전 반지름의 몇 배 크기로 정사각형 형태로 잘라낼지
def crop_coin_images(src: np.ndarray, circles, crop_scale: float = 2.5):

    coin_imgs = [] # 동전 이미지는 리스트 형태로 저장
    for center, radius in circles: # 리스트에서 각 중심위치와 반지름에 대해 반복

        # r은 잘라낼 정사각형 패치의 한 변 길이, 이미지 크기는 픽셀 단위므로 정수로
        # 최소 2는 보장(동전이 잘리지 않게)
        r = max(2, int(radius * crop_scale))

        # 마스크 이미지의 중심 좌표 설정
        cen = (r // 2, r // 2)

        # r x r 크기의 검정(0) 이미지를 생성, 이미지 자체는 컬러이므로 3채널 맞추기(,,3)
        mask = np.zeros((r, r, 3), np.uint8)

        # 방금 만든 검정 마스크에 중심 cen에 반지름 radius짜리 흰 원(255)을 채워서 그림
        cv2.circle(mask, cen, radius, (255, 255, 255), cv2.FILLED)

        # 중심 기준으로 r x r 크기의 정사각형 이미지 추출
        # 슬라이싱 이랑 동일
        coin = cv2.getRectSubPix(src, (r, r), center)

        # 동전만 남기기(배경 제거)
        # bitwise_and: 숫자를 2진수(bit)로 바꿔서 자리수별로 AND 연산을 수행
        # mask의 검은색(00000000), 흰색(11111111) 이걸 coin 이미지의 채널(B/G/R)별 픽셀마다, 똑같이 and연산 적용
        # 따라서 흰색 부분이었던 동전 부분만 남게 됨
        coin = cv2.bitwise_and(coin, mask)

        # 그렇게 추출된 동전 이미지를 coin_imgs 리스트에 저장
        coin_imgs.append(coin)

    return coin_imgs

# 실제로 모델 가중치 파일이 있는지 확인
import os

best_path = BEST_MODEL_PATH
last_path = LAST_MODEL_PATH

print("best exists:", os.path.exists(best_path))
print("last exists:", os.path.exists(last_path))

# 5) YOLO 불러와서 분류(동전 종류 예측)
from ultralytics import YOLO # Ultralytics 라이브러리에서 YOLO 클래스 가져옴

# 학습 중 가장 성능 좋았던 시점의 모델 가져옴
MODEL_PATH = BEST_MODEL_PATH
model = YOLO(MODEL_PATH)  # best.pt 파일을 읽고 가중치와 메타정보를 신경망 구조에 로드해서 모델 객체 생성

# 분류 결과 도출
def yolo_classify(coin_imgs, imgsz: int = 224): # 입력으로 coin_imgs를 리스트로 넣으면 이미지 여러 장을 한 번에 배치(batch) 로 처리
    preds = model.predict(coin_imgs, imgsz=imgsz, verbose=False) # model.predict(...)는 입력을 받아서 각 이미지가 어떤 클래스인지 확률을 계산
    return preds

value_map={}

# 6) 동전 총합 구하기
# model: YOLO 객체 → 여기서 **model.names(클래스 id → 클래스명)**을 꺼내 쓰려고 필요함
# preds: model.predict(coin_imgs)가 돌려준 결과 리스트 → 동전 하나당 예측값들 한 세트가 들어 있음
# value_map:  금액 매핑표(클래스명 → 한화 가치(KRW))
# conf_thres: 확률이 이 값보다 낮으면 unknown 처리
def tally_and_sum(model, preds, value_map: dict, conf_thres: float = 0.0):
    counts = Counter()
    total = 0
    unknown = 0

    for pred in preds: # preds 리스트 안의 동전 예측값들에 대해 하나씩 처리
        cls_id = int(pred.probs.top1) # 한 이미지에 대한 모든 클래스에 대한 점수/확률 분포중 가장 확률 높은 클래스 번호 추출
        cls_name = model.names[cls_id] # 위에서 뽑은 번호를 데이터셋에서 정한 클래스 라벨(폴더명)로 바꿈
        conf = float(pred.probs.top1conf) # top1conf: top1 클래스가 맞을 확률(신뢰도)를 conf로 저장

        # 분류 확률이 너무 낮으면 unknown으로 바꾸기
        if conf < conf_thres:
            cls_name = "unknown"

        # 만약 그렇게 나온 클래스 라벨이 value_map에서 찾을 수 있으면
        if cls_name in value_map:
            counts[cls_name] += 1 # 해당하는 클래스 라벨이 몇 개인지 누적
            total += value_map[cls_name] # total에 그 동전의 한화 가치를 더함
        else:
            unknown += 1 # 없으면 모름에 하나 추가

    # counts(클래스별 개수표)/total(한화 총액)/unknown(처리 못한 동전 수) 반환
    return dict(counts), total, unknown

### 모델

In [ ]:
!pip -q install ultralytics

from ultralytics import YOLO

## 이 부분에 증강 요소 넣기##

def train_yolo():
    model = YOLO('yolov8s-cls.pt')  # 사용할 가중치 모델 선택

    results = model.train(
        data=DATASET_PATH,  # 데이터셋 경로
        epochs=150,
        imgsz=224,
        batch=32,
        project='drive/MyDrive/yolov8_project',
        name="coins_cls",
        device=0,
        verbose=True,
        exist_ok=True,
        augment=True, # 데이터 증강
    )
    return results

if __name__ == '__main__':
    results = train_yolo()
    print("✅ 학습이 완료되었습니다.")
    print(results)
